2018/05/01

打ち合わせ資料

櫻岡響

# 列番号からの文字列推定実験
## RDBの使用により以下が実現
- sentenceをid指定で呼び出し
- 言語モデルとsentenceがidで紐付いている
- 生成した言語モデルにチェックサムが付いているので、後から再現可能（現状バイナリで保存しているので16進数文字列に変更予定）

# 準備

In [1]:
from functools import reduce
from sklearn.model_selection import train_test_split

from anakin.lang_model.lang_model import LangModel

from anakin.lang_model_file_factory import LangModelFileFactory
from anakin.db.session import database, engine_and_session
from anakin.db.model import Base, Sentence

from anakin.util.word import Word
from anakin.util.estimate import estimate
from anakin.util.key_to_word import KeyToWord
from anakin.util.dp_matching import DPMatching

from env import LOCAL

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
db = database(**LOCAL)
ENGINE, Session = engine_and_session(db, echo=False)
Base.prepare(ENGINE, reflect=True)

lmff = LangModelFileFactory(db, echo=False)

In [4]:
def id_to_vocab(Session, id_list):
    session = Session()
    query = session.query(Sentence).filter(Sentence.id.in_(id_list))
    
    result = set()
    for sentence in query:
        con = sentence.contents
        words = Word.from_sentence(con)
        result = result | set(words)
        
    session.close()
    return KeyToWord(result)

In [5]:
def id_to_lm(lang_model_file_factory, id_list, order):
    lmff = lang_model_file_factory
    lmf = lmff.get_lang_model_file(id_list, order)
    arpa = lmf.contents.decode()
    return lmf.checksum, LangModel(arpa)

# 全SentenceのID取得

In [6]:
session = Session()
all_id = [s[0] for s in session.query(Sentence.id)]

# 実験条件

In [7]:
order = 3
test_num = 30
seed = 12345678

# 実験条件をもとにトレーニングデータとテストデータに分ける

In [8]:
size = len(all_id)
size

1000

In [9]:
train, test = train_test_split(all_id, test_size=test_num/size, random_state=seed)

close_train_id = all_id
open_train_id = train

# テストセットのイテレーション例

In [10]:
# session.queryの使用例
# idがtestに含まれるSentenceをイテレート
session = Session()
query = session.query(Sentence).filter(Sentence.id.in_(test))

for sentence in query:
    print(sentence)
    
session.close()

<Sentence(id=40, contents='フロント/フロント の/ノ 対応/タイオウ が/ガ 良く/ヨク 感じ/カンジ が/ガ 良かっ/ヨカッ た/タ です/デス')
<Sentence(id=41, contents='部屋/ヘヤ は/ハ 普通/フツウ です/デス が/ガ 清潔/セイケツ 感/カン が/ガ あり/アリ まずまず/マズマズ です/デス')
<Sentence(id=165, contents='が/ガ 第/ダイ 一/イチ な/ナ ので/ノデ 朝/アサ ズボン/ズボン の/ノ プレス/プレス が/ガ でき/デキ ない/ナイ のに/ノニ 困る/コマル')
<Sentence(id=167, contents='この/コノ 全/ゼン 室/シツ ズボンプレッサー/ズボンプレッサー 付き/ツキ は/ハ 大変/タイヘン 良い/ヨイ')
<Sentence(id=210, contents='かかわら/カカワラ ず/ズ')
<Sentence(id=234, contents='使い/ツカイ やすい/ヤスイ ホテル/ホテル だっ/ダッ た/タ と/ト 言っ/イッ て/テ まし/マシ た/タ')
<Sentence(id=236, contents='弟/オトウト の/ノ 教育/キョウイク 実習/ジッシュウ の/ノ ため/タメ 予約/ヨヤク し/シ た/タ 兄/アニ です/デス')
<Sentence(id=321, contents='<num> 月/ツキ <num> 日/ニチ 一/イチ 泊/ハク し/シ まし/マシ た/タ')
<Sentence(id=351, contents='きれい/キレイ な/ナ ホテル/ホテル で/デ 心斎橋/シンサイバシ に/ニ 近い/チカイ という/トイウ 条件/ジョウケン を/ヲ 考え/カンガエ たら/タラ かなり/カナリ リーズナブル/リーズナブル な/ナ 利用/リヨウ 料金/リョウキン と/ト 思え/オモエ ます/マス')
<Sentence(id=406, contents='たまたま/タマタマ 駅/エキ 側/ガワ の/ノ 部屋/ヘヤ だっ/ダッ た/タ ので/ノデ 電車/デンシャ の/ノ 発着/ハッチャク の/ノ 音/オト が/ガ すこし/スコシ 気/キ に/ニ なり/ナリ まし/マシ た/タ')
<Sent

# クローズドテスト

In [11]:
close_train_vocab = id_to_vocab(Session, close_train_id)
checksum, close_train_lm = id_to_lm(lmff, close_train_id, order)

session = Session()
query = session.query(Sentence).filter(Sentence.id.in_(test))

print(checksum)
print()

for sentence in query:
    ref = Word.from_sentence(sentence.contents)
    print(' / '.join([w.surface for w in ref]))
    est = estimate(ref, close_train_vocab, close_train_lm, order)
    print(' / '.join([w.surface for w in est]))
    dpm = DPMatching(ref, est)
    print(dpm.score)
    print()
session.close()

b'\x97\xb6[\x82\x8e\xce\xa8\x17\x14\x8a-\x92wH\x9cO\xed\xc6\x11\xb7'

フロント / の / 対応 / が / 良く / 感じ / が / 良かっ / た / です
フロント / の / 対応 / が / 良く / 感じ / が / 良かっ / た / です
Score(perfect=10, yomi=0, miss=0, ignore=False)

部屋 / は / 普通 / です / が / 清潔 / 感 / が / あり / まずまず / です
部屋 / は / 普通 / です / が / 清潔 / 感 / が / あり / まずまず / です
Score(perfect=11, yomi=0, miss=0, ignore=False)

が / 第 / 一 / な / ので / 朝 / ズボン / の / プレス / が / でき / ない / のに / 困る
が / 第 / 一 / な / ので / 朝 / ズボン / の / プレス / が / 出来 / ない / のに / 困る
Score(perfect=13, yomi=1, miss=0, ignore=False)

この / 全 / 室 / ズボンプレッサー / 付き / は / 大変 / 良い
この / 全 / 室 / ズボンプレッサー / 付き / は / 大変 / 良い
Score(perfect=8, yomi=0, miss=0, ignore=False)

かかわら / ず
かかわら / ず
Score(perfect=2, yomi=0, miss=0, ignore=False)

使い / やすい / ホテル / だっ / た / と / 言っ / て / まし / た
使い / やすい / ホテル / だっ / た / と / 言っ / て / まし / た
Score(perfect=10, yomi=0, miss=0, ignore=False)

弟 / の / 教育 / 実習 / の / ため / 予約 / し / た / 兄 / です
弟 / の / 教育 / 実習 / の / ため / 予約 / し / た / 兄 / です
Score(perfect=11, yomi=0, miss

# オープンテスト

In [12]:
open_train_vocab = id_to_vocab(Session, open_train_id)
checksum, open_train_lm = id_to_lm(lmff, open_train_id, order)

session = Session()
query = session.query(Sentence).filter(Sentence.id.in_(test))

print(checksum)
print()

for sentence in query:
    ref = Word.from_sentence(sentence.contents)
    print(' / '.join([w.surface for w in ref]))
    est = estimate(ref, open_train_vocab, open_train_lm, order)
    print(' / '.join([w.surface for w in est]))
    dpm = DPMatching(ref, est)
    print(dpm.score)
    print()
session.close()

b"\xc493@j\xbe6E\xa0\xac\x97!'\x9d*\xb8\t\xc4\t\xbd"

フロント / の / 対応 / が / 良く / 感じ / が / 良かっ / た / です
フロント / の / 対応 / が / 良く / 感じ / が / 良かっ / た / です
Score(perfect=10, yomi=0, miss=0, ignore=False)

部屋 / は / 普通 / です / が / 清潔 / 感 / が / あり / まずまず / です
部屋 / は / ビデオ / です / が / 清潔 / 感 / が / あり / ます / ます / です
Score(perfect=9, yomi=0, miss=-1, ignore=False)

が / 第 / 一 / な / ので / 朝 / ズボン / の / プレス / が / でき / ない / のに / 困る
こと / アウト / な / ので / 味 / 至便 / 上り / 坂 / でき / ない / の / の / 困り
Score(perfect=4, yomi=0, miss=-2, ignore=False)

この / 全 / 室 / ズボンプレッサー / 付き / は / 大変 / 良い
金沢 / 市 / で / ズボン・プレッサー / とき / は / 大変 / 良い
Score(perfect=3, yomi=1, miss=0, ignore=False)

かかわら / ず
かかわら / ず
Score(perfect=2, yomi=0, miss=0, ignore=False)

使い / やすい / ホテル / だっ / た / と / 言っ / て / まし / た
近い / 安い / ホテル / で / とっ / て / あっ / た / まし / た
Score(perfect=3, yomi=1, miss=0, ignore=False)

弟 / の / 教育 / 実習 / の / ため / 予約 / し / た / 兄 / です
弟 / の / 教育 / 実習 / の / ため / 予約 / し / たい / の / です
Score(perfect=9, yomi=0, miss=0, ignore=False)